# **Installing Necessary Libraries**

---



---



In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

# **Importing Necessary Libraries**

---



---



In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset
import numpy as np

# **Loading AutoTokenizer for text processing**

---



---



In [3]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# **Importing dataset for Spam Detection from HuggingFace**

---



---



In [4]:
dataset=load_dataset("Deysi/spam-detection-dataset")

Generating train split:   0%|          | 0/8175 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2725 [00:00<?, ? examples/s]

# **Remaping of dataset By adding Three columns:**

---



---


## **'input_ids'  ,  'token_type_ids'  &  'attention_mask'**

In [5]:
def preprocess(batch):
  return tokenizer(batch['text'], truncation=True, padding=True,return_tensors="tf")
updataset=dataset.map(preprocess,batched=True)

Map:   0%|          | 0/8175 [00:00<?, ? examples/s]

Map:   0%|          | 0/2725 [00:00<?, ? examples/s]

# **Removing unnecessary columns from updataset for preparing train dataset**

---



---



In [7]:
Mails=updataset['train'].remove_columns(['text','label'])

In [8]:
labels=np.array([0 if i=='not_spam' else 1 for i in updataset['train']['label']])

# **Building A custom Subclass model with 'bert-base-uncased'**

---



---



In [45]:
class SpamOrNot(tf.keras.Model):
  def __init__(self,dff=1):
    super(SpamOrNot,self).__init__()
    self.dff=dff
    self.Bert=TFAutoModel.from_pretrained('bert-base-uncased')
    self.seq=tf.keras.Sequential([
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(dff,activation='sigmoid')
    ])

  def call(self,inputs):
    input_ids=inputs[0]
    attention_mask=inputs[1]
    token_type_ids=inputs[2]
    output=self.Bert(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
    embed=output.pooler_output
    return self.seq(embed)

# **Initializing and Compiling Model**

---



---



In [46]:
Model=SpamOrNot()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [47]:
Model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
      loss='binary_crossentropy',
      metrics=['accuracy']
    )

# **Preprocessing of dataset for training**

---



---



In [48]:
def prepare_data(mails):
    input_ids = tf.constant(mails['input_ids'])
    attention_mask = tf.constant(mails['attention_mask'])
    token_type_ids = tf.constant(mails['token_type_ids'])
    return [input_ids, attention_mask, token_type_ids]

# **Slicing Dataset to small number of Sequence**

---



---



In [49]:
inputs=prepare_data(Mails[:1000])
lable=labels[:1000]
testInput=prepare_data(Mails[1000:1200])
testlable=labels[1000:1200]

# **Training Model**

---



---



In [50]:
Model.fit(x=inputs,y=lable,epochs=2,batch_size=16)

Epoch 1/2
63/63 [==============================] - 163s 2s/step - loss: 0.2444 - accuracy: 0.9480
Epoch 2/2
63/63 [==============================] - 104s 2s/step - loss: 0.0283 - accuracy: 0.9960


# **Evaluating Model**

---



---



In [51]:
Model.evaluate(x=testInput,y=testlable)

7/7 [==============================] - 18s 1s/step - loss: 0.0157 - accuracy: 1.0000


[0.0157232116907835, 1.0]